# Imports

In [4]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from collections import defaultdict
import torch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass

In [5]:
from utils.data_retrieval import process_annotations
from utils.categorisation import categorize_facial_mot, categorize_utterance_mot, categorize_gaze_mot, categorize_gaze_chi
from utils.multimodal_dataset import MultimodalDataset
from utils.dataset_split import DatasetSplit
from utils.train_loop import train_loop
from utils.evaluation_loop import evaluation_loop

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [7]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Data Preparation

In [8]:
file_name = "A_12_20130925_1212_01_2.txt"

In [9]:
process_annotations(file_name, "annotations_per_t.csv")

Per-second annotations saved to /content/drive/MyDrive/ISIR/RedBallGame/final/annotations_per_t.csv


## Annotations categorization

In [10]:
df = pd.read_csv("annotations_per_t.csv")

In [11]:
df['Annotation'] = df.apply(
    lambda row: categorize_facial_mot(row['Annotation']) if row['Tier'] == 'Facial@MOT' else
                categorize_utterance_mot(row['Annotation']) if row['Tier'] == 'Utterance@MOT' else
                categorize_gaze_mot(row['Annotation']) if row['Tier'] == 'Gaze@MOT' else
                categorize_gaze_chi(row['Annotation']) if row['Tier'] == 'Gaze@CHI' else
                row['Annotation'],
    axis=1
)

## Create objects

### Standard

In [12]:
tiers = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI"
]

pivot_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

pivot_df = pivot_df[tiers]

pivot_df = pivot_df.where(pd.notnull(pivot_df), None)

records = pivot_df.to_dict(orient='records')

X = records.copy()

### With GazeRelation

In [13]:
tiersGR = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI",
    "GazeRelation"
]
pivot_gr_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

records_gr = pivot_gr_df.to_dict(orient='records')

for record in records_gr:
    mot_target = record.get("Gaze@MOT")
    chi_target = record.get("Gaze@CHI")

    if chi_target == "mom" and mot_target == "child":
        label = "MA"  # MutualAttention
    elif mot_target == "invisible":
        label = "<UNK>"  # Unknown
    elif chi_target == "mom":
        label = "SA(CHI)"  # SingleAttention(CHI)
    elif mot_target == "child":
        label = "SA(MOT)"  # SingleAttention(MOT)
    elif chi_target == mot_target and chi_target is not None:
        label = "ShA"  # SharedAttention
    elif chi_target != mot_target:
        label = "L"  # LostAttention
    else:
        label = "<UNK>"  # Unknown

    record["GazeRelation"] = label

pivot_gr_df = pd.DataFrame(records_gr)

pivot_gr_df = pivot_gr_df[tiersGR]

pivot_gr_df = pivot_gr_df.where(pd.notnull(pivot_gr_df), None)

records_gr = pivot_gr_df.to_dict(orient='records')

X_gr = records_gr.copy()

## Retrieve vocabs

In [14]:
values = defaultdict(set)

for r in records_gr:
    for key in r.keys():
        value = r[key]
        if value is None or value == '' or value == 'invisible':
            value = "<UNK>"  # use <UNK> token for missing
        values[key].add(value)

vocab_action    = {token: idx for idx, token in enumerate(sorted(values['Action@MOT']))}
vocab_gaze      = {token: idx for idx, token in enumerate(sorted(values['Gaze@MOT']))}
vocab_utterance = {token: idx for idx, token in enumerate(sorted(values['Utterance@MOT']))}
vocab_prosody   = {token: idx for idx, token in enumerate(sorted(values['Prosody@MOT']))}
vocab_facial    = {token: idx for idx, token in enumerate(sorted(values['Facial@MOT']))}
vocab_laughter  = {token: idx for idx, token in enumerate(sorted(values['Laughter@MOT']))}
vocab_laughter_chi = {token: idx for idx, token in enumerate(sorted(values['Laughter@CHI']))}
vocab_gaze_chi  = {token: idx for idx, token in enumerate(sorted(values['Gaze@CHI']))}
vocab_gazerelation  = {token: idx for idx, token in enumerate(sorted(values['GazeRelation']))}

In [15]:
vocabs = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
          "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
          "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

vocabs_gr = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
             "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
             "gazerelation": vocab_gazerelation, "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

# Helper functions

In [16]:
def get_data(isGazeRelation):
    if isGazeRelation:
        return X_gr, vocabs_gr
    else:
        return X, vocabs

## Data Loader (batches divsion)

In [18]:
from torch.utils.data import DataLoader

# Finding the index of a value in a vocabulary
def lookup(vocab, value):
    if value is None:
        value = "<UNK>"
    return vocab.get(value, vocab["<UNK>"])

def collate_fn(batch):
    """
    Collate function for DataLoader without GazeRelation target.
    Processes a batch of data into input sequences and target tensors.
    """
    sequences, targets_list = zip(*batch)  # Separate inputs and targets

    target_laughter_indices = []
    target_gaze_indices = []

    # Convert each target value to an index
    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    # Convert target index lists to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    # Wrap each sequence dict in a list to create a sequence of length 1
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter)

def collate_fn_gr(batch):
    """
    Collate function for DataLoader with GazeRelation target.
    Same as collate_fn but also processes the 'GazeRelation' target.
    """
    sequences, targets_list = zip(*batch)

    target_laughter_indices = []
    target_gaze_indices = []
    target_gazerelation_indices = []

    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        gazerelation_val = target_dict.get('GazeRelation')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))
        target_gazerelation_indices.append(lookup(vocab_gazerelation, gazerelation_val))

    # Convert lists to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)
    targets_gazerelation = torch.tensor(target_gazerelation_indices, dtype=torch.long)

    # Wrap each sequence dict in a list to create a sequence of length 1
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter, targets_gazerelation)

def get_loader(X, y, batch_size, isGazeRelation):
    """
    Creates a DataLoader for the given dataset.
    """
    dataset = MultimodalDataset(X, y)
    if isGazeRelation:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_gr)
    else:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return loader


In [19]:
def get_loaders(X, isGazeRelation, isMixedPhases, train_size, val_size, n_parts):
    """
    Splits the dataset into train/validation/test sets and returns DataLoaders for each.
    """
    # If phases are not mixed, process the dataset as a single part
    if not isMixedPhases:
        n_parts = 1

    # Split the dataset into train, validation, and test subsets
    X_train, y_train, X_val, y_val, X_test, y_test = DatasetSplit(
        dataset=X,
        n_parts=n_parts,
        train_size=train_size,
        val_size=val_size
    ).main()

    # Determine batch sizes so that each phase is represented in a batch
    train_batch_size = len(X_train) // n_parts
    val_batch_size = len(X_val) // n_parts
    test_batch_size = len(X_test) // n_parts

    # Create DataLoaders for each subset
    train_loader = get_loader(X_train, y_train, batch_size=train_batch_size, isGazeRelation=isGazeRelation)
    val_loader = get_loader(X_val, y_val, batch_size=val_batch_size, isGazeRelation=isGazeRelation)
    test_loader = get_loader(X_test, y_test, batch_size=test_batch_size, isGazeRelation=isGazeRelation)

    return train_loader, val_loader, test_loader


## Training Model

In [21]:
@dataclass
class DatasetParams:
    isGazeRelation: bool
    isMixedPhases: bool
    train_size: int
    val_size: int
    n_parts: int

@dataclass
class ModelParams:
    hidden_dim: int
    num_layers: int
    dropout: float
    input_size: int
    bidirectional: bool

@dataclass
class TrainParams:
    lr: float
    weight_decay: float
    num_epochs: int

In [30]:
def training_model(model, dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    """
    Trains, validates, and evaluates the given model.
    """
    # Load dataset and corresponding vocabularies
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    model = model(
        vocabs,
        dataset_params.isGazeRelation,
        hidden_dim=model_params.hidden_dim,
        num_layers=model_params.num_layers,
        dropout=model_params.dropout,
        bidirectional=model_params.bidirectional,
        input_size=model_params.input_size
    ).to(device)

    train_loader, val_loader, test_loader = get_loaders(
        dataset,
        dataset_params.isGazeRelation,
        dataset_params.isMixedPhases,
        dataset_params.train_size,
        dataset_params.val_size,
        dataset_params.n_parts
    )

    optimizer = optim.Adam(model.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
    criterion = nn.CrossEntropyLoss()

    num_epochs = train_params.num_epochs

    print("Starting training...")
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        # ---------------- TRAINING ----------------
        model.train()
        model = train_loop(model, train_loader, criterion, optimizer, dataset_params.isGazeRelation, device)

        model.eval()
        with torch.no_grad():
            avg_train_loss, loss_gaze, loss_laughter, train_gaze_accuracy, train_gaze_precision, train_gaze_f1, train_laughter_accuracy, train_laughter_precision, train_laughter_f1 = evaluation_loop(
                model, train_loader, criterion, dataset_params.isGazeRelation, device
            )

        # ---------------- VALIDATION ----------------
        model.eval()
        with torch.no_grad():
            avg_val_loss, loss_gaze, loss_laughter, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                model, val_loader, criterion, dataset_params.isGazeRelation, device
            )

        print(
            f"Epoch [{epoch+1}/{num_epochs}], "
            f"\tTrain Loss: {avg_train_loss:.4f}, Train Gaze Acc: {train_gaze_accuracy:.4f}, Train Gaze Prec: {train_gaze_precision:.4f}, Train Gaze F1: {train_gaze_f1:.4f}, Train Laughter Acc: {train_laughter_accuracy:.4f}, Train Laughter Prec: {train_laughter_precision:.4f}, Train Laughter F1: {train_laughter_f1:.4f}, "
            f"\n\tVal Loss: {avg_val_loss:.4f}, Val Gaze Acc: {val_gaze_accuracy:.4f}, Val Gaze Prec: {val_gaze_precision:.4f}, Val Gaze F1: {val_gaze_f1:.4f}, Val Laughter Acc: {val_laughter_accuracy:.4f}, Val Laughter Prec: {val_laughter_precision:.4f}, Val Laughter F1: {val_laughter_f1:.4f}"
        )

    print("Training finished.")

    # ---------------- TEST EVALUATION ----------------
    model.eval()
    with torch.no_grad():
        avg_test_loss, loss_gaze, loss_laughter, test_accuracy_gaze, test_gaze_precision, test_gaze_f1, test_accuracy_laughter, test_laughter_precision, test_laughter_f1 = evaluation_loop(
            model, test_loader, criterion, dataset_params.isGazeRelation, device
        )

    print("\nTest Set Evaluation:")
    print(
        f"Test Loss: {avg_test_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Gaze Prec: {test_gaze_precision:.4f}, Test Gaze F1: {test_gaze_f1:.4f}, "
        f"Test Laughter Acc: {test_accuracy_laughter:.4f}, Test Laughter Prec: {test_laughter_precision:.4f}, Test Laughter F1: {test_laughter_f1:.4f}"
    )

    return model


# Single LSTM

## Model

In [ ]:
class EnfantReactionPredictor(nn.Module):
    """
      Neural network to predict child reactions (gaze, laughter, optionally gaze relation)

      Args:
          vocabs: dictionary of vocabularies for all modalities
          isGazeRelation: boolean - if True, use GazeRelation in addition to gaze/laughter
          hidden_dim
          num_layers
          dropout
          input_size
          bidirectional: if True, use bidirectional LSTM
    """
    def __init__(self,
                 vocabs,
                 isGazeRelation,
                 hidden_dim=16,
                 num_layers=2,
                 dropout=0.0,
                 input_size=40,
                 bidirectional=False):

        super().__init__()

        self.isGazeRelation = isGazeRelation

        # ---------------- VOCABS ----------------
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_gaze_chi = vocabs["gaze_chi"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # ---------------- EMBEDDINGS ----------------
        # Embedding layers for mother features
        self.embed_gaze = nn.Embedding(len(vocabs["gaze"]), 8)
        self.embed_utterance = nn.Embedding(len(vocabs["utterance"]), 8)
        self.embed_prosody = nn.Embedding(len(vocabs["prosody"]), 4)
        self.embed_facial = nn.Embedding(len(vocabs["facial"]), 8)
        self.embed_laughter = nn.Embedding(len(vocabs["laughter"]), 4)

        # Sentence encoder for action text using a pre-trained transformer
        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        # Embeddings for child targets
        self.embed_laughter_chi = nn.Embedding(len(vocabs["laughter_chi"]), 4)
        self.embed_gaze_chi = nn.Embedding(len(vocabs["gaze_chi"]), 8)

        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # ---------------- LSTM ----------------
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )

        self.dropout = nn.Dropout(dropout)

        # Determine input size for decoder based on bidirectional LSTM
        decoder_input_dim = hidden_dim * (2 if bidirectional else 1)

        # ---------------- DECODERS ----------------
        # Linear layers to predict child gaze and laughter
        self.decoder_gaze = nn.Linear(decoder_input_dim, len(vocabs["gaze_chi"]))
        self.decoder_laughter = nn.Linear(decoder_input_dim, len(vocabs["laughter_chi"]))

        # Decoder for GazeRelation
        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(decoder_input_dim, len(self.vocab_gazerelation))

    # ---------------- LOOKUP ----------------
    def lookup(self, vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    # ---------------- ENCODE INPUTS ----------------
    def encode_input_batch(self, batch_sequence_of_dicts, device):
        """
        Convert a batch of sequences of dictionaries into embeddings concatenated as LSTM input.

        """
        all_seqs = []

        for sequence in batch_sequence_of_dicts:
            seq = []
            for step_dict in sequence:
                # Convert mother features to indices
                gaze = torch.tensor([self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT"))], dtype=torch.long)
                utterance = torch.tensor([self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT"))], dtype=torch.long)
                prosody = torch.tensor([self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT"))], dtype=torch.long)
                facial = torch.tensor([self.lookup(self.vocab_facial, step_dict.get("Facial@MOT"))], dtype=torch.long)
                laughter = torch.tensor([self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT"))], dtype=torch.long)

                # Encode action text using sentence transformer
                action_text = step_dict.get("Action@MOT") or "<UNK>"
                encoded_action = torch.tensor(self.sentence_encoder.encode(action_text), dtype=torch.float32).unsqueeze(0)

                # Concatenate embeddings and action encoding
                step_emb = torch.cat([
                    self.embed_gaze(gaze),
                    self.embed_utterance(utterance),
                    self.embed_prosody(prosody),
                    self.embed_facial(facial),
                    self.embed_laughter(laughter),
                    encoded_action
                ], dim=-1)

                seq.append(step_emb)

            seq = torch.cat(seq, dim=0)
            all_seqs.append(seq)

        batch_tensor = torch.stack(all_seqs).to(device)
        return batch_tensor

    # ---------------- FORWARD PASS ----------------
    def forward(self, batch_sequence_of_dicts, device):

        # Encode batch sequences
        encoded_steps = self.encode_input_batch(batch_sequence_of_dicts, device)

        # Run through LSTM
        output, (h_n, c_n) = self.lstm(encoded_steps)

        # Apply decoders for each target
        logits_gaze = self.decoder_gaze(output.squeeze(1))
        logits_laughter = self.decoder_laughter(output.squeeze(1))

        if self.isGazeRelation:
            logits_gazerelation = self.decoder_gazerelation(output.squeeze(1))
            return logits_gaze, logits_laughter, logits_gazerelation

        return logits_gaze, logits_laughter


## Grid Search

In [26]:
from sklearn.model_selection import ParameterGrid

def grid_search(model, dataset_params: DatasetParams, param_grid: dict):

    best_val_loss = float('inf')
    best_params = None
    results = []

    # Load dataset and vocabularies
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    # Convert param_grid dict into a list of all combinations
    param_grid = ParameterGrid(param_grid)

    print(f"Starting Grid Search with {len(param_grid)} combinations...")

    # Iterate over all hyperparameter combinations
    for i, params in enumerate(tqdm(param_grid, desc="Processing params", unit="set")):
        print(f"\n--- Running combination {i+1}/{len(param_grid)} ---")
        print(f"Parameters: {params}")

        # Create ModelParams and TrainParams dataclasses for the current combination
        model_params = ModelParams(
            hidden_dim=params['hidden_dim'],
            num_layers=params['num_layers'],
            dropout=params['dropout'],
            bidirectional=params['bidirectional'],
            input_size=416
        )
        train_params = TrainParams(
            lr=params['lr'],
            weight_decay=params['weight_decay'],
            num_epochs=params['num_epochs']
        )

        model_instance = model(
            vocabs,
            dataset_params.isGazeRelation,
            hidden_dim=model_params.hidden_dim,
            num_layers=model_params.num_layers,
            dropout=model_params.dropout,
            bidirectional=model_params.bidirectional,
            input_size=model_params.input_size
        ).to(device)

        train_loader, val_loader, test_loader = get_loaders(
            dataset,
            dataset_params.isGazeRelation,
            dataset_params.isMixedPhases,
            dataset_params.train_size,
            dataset_params.val_size,
            dataset_params.n_parts
        )

        optimizer = optim.Adam(model_instance.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
        criterion = nn.CrossEntropyLoss()

        # ---------------- TRAINING LOOP ----------------
        for epoch in tqdm(range(train_params.num_epochs), desc=f"Training Epochs (Params {params})"):
            model_instance.train()

            model_instance = train_loop(model_instance, train_loader, criterion, optimizer, dataset_params.isGazeRelation, device)

            model_instance.eval()
            with torch.no_grad():
                avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                    model_instance, val_loader, criterion, dataset_params.isGazeRelation, device
                )

        # ---------------- VALIDATION EVALUATION ----------------
        # Evaluate model on test set after training
        model_instance.eval()
        with torch.no_grad():
            avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                model_instance, test_loader, criterion, dataset_params.isGazeRelation, device
            )

        print(f"Validation Loss for this combo: {avg_val_loss:.4f}")

        results.append({
            'params': params,
            'val_loss': avg_val_loss,
            'val_gaze_accuracy': val_gaze_accuracy,
            'val_gaze_precision': val_gaze_precision,
            'val_gaze_f1': val_gaze_f1,
            'val_laughter_accuracy': val_laughter_accuracy,
            'val_laughter_precision': val_laughter_precision,
            'val_laughter_f1': val_laughter_f1,
        })

        # Update best parameters if this combination has lowest validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_params = params

    print("\n--- Grid Search Finished ---")
    print(f"Best parameters found: {best_params}")
    print(f"Best validation loss: {best_val_loss:.4f}")

    return results, best_params

## Grid Search Results

### Results - Standard

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results:
      print(res)

results_df = pd.DataFrame(results)

print("\nGrid Search Results DataFrame:")
display(results_df)

results_df.to_csv('grid_search_lstm_results_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.4494

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.7133

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 1.2808

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.3694685697555542, 'val_gaze_accuracy': np.float64(0.6064981949458483), 'val_gaze_precision': 0.6876587184858345, 'val_gaze_f1': 0.5249617161721121, 'val_laughter_accuracy': np.float64(0.7196149217809867), 'val_laughter_precision': 0.7277550393381101, 'val_laughter_f1': 0.7192453340684299}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.5380584001541138, 'val_gaze_accuracy': np.float64(0.5186522262334536), 'val_gaze_precision': 0.2690001317769176, 'val_gaze_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.369469,0.606498,0.687659,0.524962,0.719615,0.727755,0.719245
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.538058,0.518652,0.269000,0.354262,0.469314,0.220256,0.299808
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.328139,0.752106,0.699334,0.721343,0.743682,0.744911,0.743908
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.381950,0.752106,0.699334,0.721343,0.750903,0.751627,0.751081
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.376313,0.596871,0.649981,0.517669,0.712395,0.726244,0.710983
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.612409,0.518652,0.269000,0.354262,0.469314,0.220256,0.299808
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.327551,0.735259,0.693899,0.699766,0.737665,0.739830,0.737910
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.323083,0.766546,0.712622,0.736461,0.725632,0.725923,0.725739
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.348791,0.735259,0.693899,0.699766,0.752106,0.762960,0.751494
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.280839,0.765343,0.711509,0.735209,0.762936,0.762734,0.762701


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results2:
      print(res)

results_df_2 = pd.DataFrame(results2)

print("\nGrid Search Results DataFrame:")
display(results_df_2)


results_df_2.to_csv('grid_search_lstm_results_df_2.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.2338

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.2077

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 1.1959

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.2303683757781982, 'val_gaze_accuracy': np.float64(0.8508064516129032), 'val_gaze_precision': 0.8255766897107633, 'val_gaze_f1': 0.8354062547326973, 'val_laughter_accuracy': np.float64(0.7036290322580645), 'val_laughter_precision': 0.7155675929658571, 'val_laughter_f1': 0.7026243522020306}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.2141703367233276, 'val_gaze_accuracy': np.float64(0.8508064516129032), 'val_gaze_precision': 0.8255766897107633, 'val_gaze

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.230368,0.850806,0.825577,0.835406,0.703629,0.715568,0.702624
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.214170,0.850806,0.825577,0.835406,0.691532,0.701587,0.690798
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.201738,0.850806,0.825577,0.835406,0.691532,0.700851,0.690939
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.217778,0.836694,0.808973,0.821922,0.687500,0.696045,0.687031
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.248717,0.725806,0.725961,0.698479,0.681452,0.683275,0.677336
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.215792,0.836694,0.808973,0.821922,0.707661,0.721484,0.706336
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.217737,0.836694,0.808973,0.821922,0.691532,0.700851,0.690939
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.195930,0.850806,0.825577,0.835406,0.703629,0.715568,0.702624
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.250528,0.737903,0.720104,0.715783,0.679435,0.683823,0.679611
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.199151,0.850806,0.825577,0.835406,0.703629,0.715568,0.702624


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results3:
      print(res)

results_df_3 = pd.DataFrame(results3)

print("\nGrid Search Results DataFrame:")
display(results_df_3)

results_df_3.to_csv('grid_search_lstm_results_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.4916

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.4630

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 1.4630

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.5424147182040744, 'val_gaze_accuracy': np.float64(0.7398496240601504), 'val_gaze_precision': 0.7711753350600812, 'val_gaze_f1': 0.7295314943527822, 'val_laughter_accuracy': np.float64(0.6962406015037594), 'val_laughter_precision': 0.7191288209954061, 'val_laughter_f1': 0.6821436156978402}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.5060040288501315, 'val_gaze_accuracy': np.float64(0.7293233082706767), 'val_gaze_precision': 0.7735304588717618, 'val_gaze_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.542415,0.739850,0.771175,0.729531,0.696241,0.719129,0.682144
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.506004,0.729323,0.773530,0.717550,0.694737,0.712133,0.682846
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.532812,0.738346,0.763168,0.728683,0.682707,0.707664,0.665872
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.504263,0.729323,0.773530,0.717550,0.708271,0.722142,0.699210
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.541532,0.736842,0.769498,0.726389,0.687218,0.705626,0.674002
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.565669,0.736842,0.769498,0.726389,0.684211,0.689423,0.677946
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.519800,0.732331,0.775098,0.720751,0.696241,0.720187,0.681708
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.495843,0.733835,0.767828,0.723237,0.711278,0.730685,0.700217
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.502615,0.738346,0.778250,0.727124,0.684211,0.710052,0.667212
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.506449,0.732331,0.775098,0.720751,0.717293,0.734767,0.707517


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results4:
      print(res)

results_df_4 = pd.DataFrame(results4)

print("\nGrid Search Results DataFrame:")
display(results_df_4)


results_df_4.to_csv('grid_search_lstm_results_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.3448

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.4192

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.3287

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.3357165389590793, 'val_gaze_accuracy': np.float64(0.7590799031476998), 'val_gaze_precision': 0.7498474188961938, 'val_gaze_f1': 0.7525137054444433, 'val_laughter_accuracy': np.float64(0.6755447941888619), 'val_laughter_precision': 0.6749771651608887, 'val_laughter_f1': 0.6752252655339483}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.405174069934421, 'val_gaze_accuracy': np.float64(0.7445520581113801), 'val_gaze_precision': 0.7405345064738714, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.335717,0.759080,0.749847,0.752514,0.675545,0.674977,0.675225
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.405174,0.744552,0.740535,0.738694,0.675545,0.676648,0.675998
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.337677,0.754237,0.745895,0.747752,0.673123,0.670962,0.671145
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.411610,0.748184,0.741007,0.741785,0.675545,0.676648,0.675998
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.376335,0.748184,0.742361,0.741769,0.671913,0.670203,0.670621
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.399620,0.742131,0.736174,0.735798,0.675545,0.676648,0.675998
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.367057,0.750605,0.745225,0.744651,0.682809,0.681778,0.682149
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.425837,0.748184,0.741007,0.741785,0.680387,0.681972,0.680985
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.332966,0.745763,0.741469,0.739887,0.673123,0.670962,0.671145
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.426790,0.745763,0.740452,0.739368,0.680387,0.681972,0.680985


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results5:
      print(res)

results_df_5 = pd.DataFrame(results5)

print("\nGrid Search Results DataFrame:")
display(results_df_5)


results_df_5.to_csv('grid_search_lstm_results_df_5.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.2826

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.3651

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.2669

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.2764714856942494, 'val_gaze_accuracy': np.float64(0.7874396135265701), 'val_gaze_precision': 0.7408996775581214, 'val_gaze_f1': 0.763459706677106, 'val_laughter_accuracy': np.float64(0.7391304347826086), 'val_laughter_precision': 0.7360511349058971, 'val_laughter_f1': 0.7368374636211307}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.2870227545499802, 'val_gaze_accuracy': np.float64(0.785024154589372), 'val_gaze_precision': 0.7385209888608948, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.276471,0.787440,0.740900,0.763460,0.739130,0.736051,0.736837
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.287023,0.785024,0.738521,0.761062,0.694444,0.697769,0.695815
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.297027,0.785024,0.738521,0.761062,0.731884,0.729933,0.730695
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.330577,0.787440,0.740900,0.763460,0.717391,0.722916,0.719336
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.266906,0.787440,0.740900,0.763460,0.711353,0.711185,0.711268
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.273086,0.787440,0.740900,0.763460,0.701691,0.703346,0.702435
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.281484,0.778986,0.732488,0.754392,0.713768,0.713937,0.713851
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.325699,0.787440,0.740900,0.763460,0.714976,0.720073,0.716821
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.281666,0.776570,0.730249,0.751960,0.731884,0.729467,0.730321
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.311198,0.785024,0.738521,0.761062,0.700483,0.702734,0.701461


### Results - with Gaze Relation

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results_gr, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr:
      print(res)

results_gr_df = pd.DataFrame(results_gr)

print("\nGrid Search Results DataFrame:")
display(results_gr_df)

results_gr_df.to_csv('grid_search_lstm_results_gr_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.3741

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.7288

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.3741

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.7297813892364502, 'val_gaze_accuracy': np.float64(0.5186522262334536), 'val_gaze_precision': 0.2690001317769176, 'val_gaze_f1': 0.35426166324345254, 'val_laughter_accuracy': np.float64(0.4693140794223827), 'val_laughter_precision': 0.22025570514407852, 'val_laughter_f1': 0.2998075200241265}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.086127996444702, 'val_gaze_accuracy': np.float64(0.5186522262334536), 'val_gaze_precision': 0.2690001317769176, 'val_gaze

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.729781,0.518652,0.269000,0.354262,0.469314,0.220256,0.299808
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.086128,0.518652,0.269000,0.354262,0.469314,0.220256,0.299808
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.473689,0.551143,0.643805,0.428686,0.707581,0.719681,0.706455
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.609653,0.518652,0.269000,0.354262,0.642599,0.748079,0.611734
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.579281,0.548736,0.640249,0.424068,0.711191,0.718596,0.710906
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.935654,0.518652,0.269000,0.354262,0.469314,0.220256,0.299808
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.444147,0.606498,0.687659,0.524962,0.722022,0.722773,0.722221
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.530565,0.551143,0.643805,0.428686,0.729242,0.729074,0.729132
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.554034,0.518652,0.269000,0.354262,0.678700,0.748762,0.662388
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.465134,0.606498,0.687659,0.524962,0.740072,0.741590,0.740313


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results_gr2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr2:
      print(res)

results_gr_df_2 = pd.DataFrame(results_gr2)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_2)

results_gr_df_2.to_csv('grid_search_lstm_results_gr_df_2.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.1693

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.2808

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.1693

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.2107496857643127, 'val_gaze_accuracy': np.float64(0.844758064516129), 'val_gaze_precision': 0.8203366871457801, 'val_gaze_f1': 0.8291775033983572, 'val_laughter_accuracy': np.float64(0.7036290322580645), 'val_laughter_precision': 0.7155675929658571, 'val_laughter_f1': 0.7026243522020306}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.2170690298080444, 'val_gaze_accuracy': np.float64(0.844758064516129), 'val_gaze_precision': 0.8203366871457801, 'val_gaze_f1

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.210750,0.844758,0.820337,0.829178,0.703629,0.715568,0.702624
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.217069,0.844758,0.820337,0.829178,0.707661,0.721484,0.706336
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.196817,0.850806,0.825577,0.835406,0.693548,0.694812,0.690441
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.220117,0.850806,0.825577,0.835406,0.693548,0.703273,0.692890
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.205739,0.844758,0.820337,0.829178,0.703629,0.715568,0.702624
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.230782,0.850806,0.825577,0.835406,0.707661,0.721484,0.706336
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.195060,0.844758,0.820337,0.829178,0.693548,0.703273,0.692890
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.189320,0.850806,0.825577,0.835406,0.703629,0.715568,0.702624
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.215064,0.709677,0.716803,0.678454,0.687500,0.688479,0.687764
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.175613,0.844758,0.820337,0.829178,0.703629,0.715568,0.702624


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results_gr3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr3:
      print(res)

results_gr_df_3 = pd.DataFrame(results_gr3)
# Store metrics for this combination
        results.append({
            'params': params,
            'val_loss': avg_val_loss,
            'val_gaze_accuracy': val_gaze_accuracy,
            'val_gaze_precision': val_gaze_precision,
            'val_gaze_f1': val_gaze_f1,
            'val_laughter_accuracy': val_laughter_accuracy,
            'val_laughter_precision': val_laughter_precision,
            'val_laughter_f1': val_laughter_f1,
        })

        # Update best parameters if this combination has lowest validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_params = params

    print("\n--- Grid Search Finished ---")
    print(f"Best parameters found: {best_params}")
    print(f"Best validation loss: {best_val_loss:.4f}")

    return results, best_params
print("\nGrid Search Results DataFrame:")
display(results_gr_df_3)

results_gr_df_3.to_csv('grid_search_results_lstm_gr_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.4973

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.5822

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.4973

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.5785196953349643, 'val_gaze_accuracy': np.float64(0.7353383458646616), 'val_gaze_precision': 0.7614075180055104, 'val_gaze_f1': 0.7255680111585573, 'val_laughter_accuracy': np.float64(0.6932330827067669), 'val_laughter_precision': 0.709977252688951, 'val_laughter_f1': 0.681480198983259}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.7044389777713351, 'val_gaze_accuracy': np.float64(0.7203007518796992), 'val_gaze_precision': 0.7572132510452888, 'val_gaze_f1

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.578520,0.735338,0.761408,0.725568,0.693233,0.709977,0.681480
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.704439,0.720301,0.757213,0.709327,0.678195,0.684765,0.670596
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.544768,0.744361,0.767007,0.736569,0.684211,0.708949,0.667697
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.571343,0.729323,0.773530,0.717550,0.679699,0.685239,0.672904
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.564825,0.736842,0.777459,0.725534,0.685714,0.703435,0.672646
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.664941,0.733835,0.760529,0.724008,0.673684,0.680586,0.665487
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.515045,0.733835,0.767828,0.723237,0.697744,0.721442,0.683501
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.629747,0.729323,0.773530,0.717550,0.678195,0.684765,0.670596
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.522160,0.738346,0.763168,0.728683,0.696241,0.719129,0.682144
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.573338,0.739850,0.779042,0.728711,0.705263,0.720345,0.695423


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results_gr4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr4:
      print(res)

results_gr_df_4 = pd.DataFrame(results_gr4)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_4)

results_gr_df_4.to_csv('grid_search_results_lstm_gr_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.3622

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.4318

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.3351

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.3350550267431471, 'val_gaze_accuracy': np.float64(0.738498789346247), 'val_gaze_precision': 0.7323761337017518, 'val_gaze_f1': 0.7326736610390087, 'val_laughter_accuracy': np.float64(0.6803874092009685), 'val_laughter_precision': 0.6796602440171108, 'val_laughter_f1': 0.6799598159526902}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.371689862675137, 'val_gaze_accuracy': np.float64(0.7530266343825666), 'val_gaze_precision': 0.7449126160087707, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.335055,0.738499,0.732376,0.732674,0.680387,0.679660,0.679960
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.371690,0.753027,0.744913,0.746560,0.668281,0.668707,0.668478
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.343497,0.739709,0.736818,0.733914,0.667070,0.667388,0.667220
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.421365,0.751816,0.743933,0.745367,0.664649,0.665421,0.664987
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.351713,0.740920,0.732251,0.734595,0.667070,0.667388,0.667220
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.405186,0.738499,0.732376,0.732674,0.675545,0.676648,0.675998
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.351078,0.739709,0.736818,0.733914,0.667070,0.667388,0.667220
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.423579,0.744552,0.738101,0.738195,0.673123,0.673996,0.673496
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.336208,0.745763,0.741469,0.739887,0.667070,0.667388,0.667220
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.382241,0.745763,0.741469,0.739887,0.675545,0.676648,0.675998


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results_gr5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr5:
      print(res)

results_gr_df_5 = pd.DataFrame(results_gr5)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_5)

results_gr_df_5.to_csv('grid_search_results_lstm_gr_df_5.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.7760

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.8046

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.7132

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.7131557663281758, 'val_gaze_accuracy': np.float64(0.5495169082125604), 'val_gaze_precision': 0.4490445292681259, 'val_gaze_f1': 0.43332693844489584, 'val_laughter_accuracy': np.float64(0.717391304347826), 'val_laughter_precision': 0.7136713612515481, 'val_laughter_f1': 0.714687728699359}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.798095703125, 'val_gaze_accuracy': np.float64(0.5591787439613527), 'val_gaze_precision': 0.4188430055656055, 'val_gaze_f1':

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.713156,0.549517,0.449045,0.433327,0.717391,0.713671,0.714688
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.798096,0.559179,0.418843,0.467298,0.710145,0.714845,0.711900
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.752682,0.539855,0.434860,0.426916,0.711353,0.712962,0.712073
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.787000,0.577295,0.509892,0.500411,0.713768,0.716938,0.715052
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.722222,0.549517,0.449045,0.433327,0.692029,0.698553,0.694328
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.802274,0.566425,0.426330,0.473222,0.687198,0.689325,0.688140
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.745400,0.539855,0.424206,0.426904,0.713768,0.715746,0.714630
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.859795,0.590580,0.719601,0.526685,0.707729,0.717888,0.710641
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.732343,0.549517,0.449045,0.433327,0.729469,0.727021,0.727892
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.772496,0.584541,0.517469,0.506415,0.698068,0.700333,0.699054


## Training

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=False,
    num_layers=1,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 15
)

model = training_model(EnfantReactionPredictor, dataset_params, model_params, train_params)

In [ ]:
# Test Loss: 2.7420, Test Gaze Acc: 0.4802, Test Gaze Prec: 0.2949, Test Gaze F1: 0.3654, Test Laughter Acc: 0.5556, Test Laughter Prec: 0.6556, Test Laughter F1: 0.5508
# Test Loss: 2.6006, Test Gaze Acc: 0.4683, Test Gaze Prec: 0.2919, Test Gaze F1: 0.3596, Test Laughter Acc: 0.5635, Test Laughter Prec: 0.6374, Test Laughter F1: 0.5656

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=False,
    num_layers=2,
    dropout=0.3,
    input_size=416
)

model = training_model(EnfantReactionPredictor,
               dataset_params,
               model_params,
               TrainParams(lr = 0.003, weight_decay = 1e-5, num_epochs = 15)
               )

Starting training...


Training Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch [1/15], 	Train Loss: 3.5472, Train Gaze Acc: 0.5125, Train Gaze Prec: 0.2675, Train Gaze F1: 0.3515, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 3.6670, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [2/15], 	Train Loss: 2.2572, Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2646, Train Gaze F1: 0.3494, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.8405, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [3/15], 	Train Loss: 1.9084, Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.7813, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Ac

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=False,
    num_layers=2,
    dropout=0.3,
    input_size=416
)

model = training_model(EnfantReactionPredictor,
               dataset_params,
               model_params,
               TrainParams(lr = 0.003, weight_decay = 1e-5, num_epochs = 15)
               )

Starting training...


Training Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch [1/15], 	Train Loss: 2.0978, Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.2059, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [2/15], 	Train Loss: 1.8073, Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.1791, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Epoch [3/15], 	Train Loss: 1.6320, Train Gaze Acc: 0.5116, Train Gaze Prec: 0.2795, Train Gaze F1: 0.3586, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.1800, Val Gaze Acc: 0.5222, Val Gaze Prec: 0.2870, Val Gaze F1: 0.3704, Val Laughter Ac

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=True,
    num_layers=2,
    dropout=0.5,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 15
)

model = training_model(EnfantReactionPredictor, dataset_params, model_params, train_params)

Starting training...


Training Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch [1/15], 	Train Loss: 2.1260, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 2.2010, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4693, Val Laughter Prec: 0.2203, Val Laughter F1: 0.2998
Epoch [2/15], 	Train Loss: 1.9069, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 2.0865, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4693, Val Laughter Prec: 0.2203, Val Laughter F1: 0.2998
Epoch [3/15], 	Train Loss: 1.6863, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 2.0507, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Ac

In [ ]:
# Test Loss: 2.8709, Test Gaze Acc: 0.2103, Test Gaze Prec: 0.0442, Test Gaze F1: 0.0731, Test Laughter Acc: 0.8341, Test Laughter Prec: 0.8943, Test Laughter F1: 0.8564

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=True,
    num_layers=2,
    dropout=0.5,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 15
)

model = training_model(EnfantReactionPredictor, dataset_params, model_params, train_params)

Starting training...


Training Epochs:   0%|          | 0/15 [00:00<?, ?it/s]

Epoch [1/15], 	Train Loss: 3.7749, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6627, Train Laughter Prec: 0.7405, Train Laughter F1: 0.5711, 
	Val Loss: 3.8109, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.5415, Val Laughter Prec: 0.7302, Val Laughter F1: 0.4490
Epoch [2/15], 	Train Loss: 3.3192, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 3.4320, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4693, Val Laughter Prec: 0.2203, Val Laughter F1: 0.2998
Epoch [3/15], 	Train Loss: 2.5939, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 2.8611, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Ac

## 10-fold CV

In [ ]:
def get_targets(y):
    target_laughter_indices = []
    target_gaze_indices = []

    for target_dict in y:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    return targets_gaze, targets_laughter

In [27]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, f1_score

def CrossValidation(dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    """
    Performs 10-fold cross-validation on the dataset.
    """
    dataset, vocabs = get_data(dataset_params.isGazeRelation)
    X_data = dataset
    y_data = dataset

    kf = KFold(n_splits=10, shuffle=False)

    fold_results = []

    print("Starting 10-Fold Cross-Validation...")

    for fold, (train_index, test_index) in enumerate(kf.split(X_data)):
        print(f"\n--- Fold {fold+1}/10 ---")

        X_train_fold = [X_data[i] for i in train_index]
        y_train_fold = [y_data[i] for i in train_index]
        X_test_fold = [X_data[i] for i in test_index]
        y_test_fold = [y_data[i] for i in test_index]

        print(f"Train Fold Size: {len(X_train_fold)}, Test Fold Size: {len(X_test_fold)}")
        print(f"X Train Fold Size: {len(X_train_fold)}, Y Train Fold Size: {len(y_train_fold)}")
        print(f"X Test Fold Size: {len(X_test_fold)}, Y Test Fold Size: {len(y_test_fold)}")

        # Create DataLoaders for the current fold
        train_loader = get_loader(X_train_fold, y_train_fold, batch_size=len(X_train_fold), isGazeRelation=dataset_params.isGazeRelation)
        test_loader = get_loader(X_test_fold, y_test_fold, batch_size=len(X_test_fold), isGazeRelation=dataset_params.isGazeRelation)

        # Initialize model for the current fold
        model_instance = EnfantReactionPredictor(
            vocabs,
            dataset_params.isGazeRelation,
            hidden_dim=model_params.hidden_dim,
            num_layers=model_params.num_layers,
            dropout=model_params.dropout,
            bidirectional=model_params.bidirectional,
            input_size=model_params.input_size
        ).to(device)

        optimizer = optim.Adam(model_instance.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
        criterion = nn.CrossEntropyLoss()

        # Training loop for the current fold
        for epoch in tqdm(range(train_params.num_epochs), desc=f"Training Epochs (Fold {fold+1})"):
            model_instance.train()
            model_instance = train_loop(model_instance, train_loader, criterion, optimizer, dataset_params.isGazeRelation, device)

        # Evaluation on the test set for the current fold
        model_instance.eval()
        with torch.no_grad():
            avg_test_loss, loss_gaze, loss_laughter, test_accuracy_gaze, test_gaze_precision, test_gaze_f1, test_accuracy_laughter, test_laughter_precision, test_laughter_f1 = evaluation_loop(
                model_instance, test_loader, criterion, dataset_params.isGazeRelation, device
            )

        print(
            f"Fold {fold+1} - "
            f"Test Loss: {avg_test_loss:.4f}, "
            f"Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Gaze Prec: {test_gaze_precision:.4f}, Test Gaze F1: {test_gaze_f1:.4f}, "
            f"Test Laughter Acc: {test_accuracy_laughter:.4f}, Test Laughter Prec: {test_laughter_precision:.4f}, Test Laughter F1: {test_laughter_f1:.4f}"
        )

        fold_results.append({
            'fold': fold + 1,
            'test_loss': avg_test_loss,
            'test_gaze_accuracy': test_accuracy_gaze,
            'test_gaze_precision': test_gaze_precision,
            'test_gaze_f1': test_gaze_f1,
            'test_laughter_accuracy': test_accuracy_laughter,
            'test_laughter_precision': test_laughter_precision,
            'test_laughter_f1': test_laughter_f1,
        })

    # Calculate and print average metrics
    avg_test_loss = sum([r['test_loss'] for r in fold_results]) / len(fold_results)
    avg_test_gaze_accuracy = sum([r['test_gaze_accuracy'] for r in fold_results]) / len(fold_results)
    avg_test_gaze_precision = sum([r['test_gaze_precision'] for r in fold_results]) / len(fold_results)
    avg_test_gaze_f1 = sum([r['test_gaze_f1'] for r in fold_results]) / len(fold_results)
    avg_test_laughter_accuracy = sum([r['test_laughter_accuracy'] for r in fold_results]) / len(fold_results)
    avg_test_laughter_precision = sum([r['test_laughter_precision'] for r in fold_results]) / len(fold_results)
    avg_test_laughter_f1 = sum([r['test_laughter_f1'] for r in fold_results]) / len(fold_results)

    print("\n--- Cross-Validation Results ---")
    print(f"Average Test Loss: {avg_test_loss:.4f}")
    print(f"Average Test Gaze Accuracy: {avg_test_gaze_accuracy:.4f}")
    print(f"Average Test Gaze Precision: {avg_test_gaze_precision:.4f}")
    print(f"Average Test Gaze F1: {avg_test_gaze_f1:.4f}")
    print(f"Average Test Laughter Accuracy: {avg_test_laughter_accuracy:.4f}")
    print(f"Average Test Laughter Precision: {avg_test_laughter_precision:.4f}")
    print(f"Average Test Laughter F1: {avg_test_laughter_f1:.4f}")

    return fold_results

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=False,
    num_layers=2,
    dropout=0.3,
    input_size=416
)
train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 5
)

CrossValidation(dataset_params, model_params, train_params)

Starting 10-Fold Cross-Validation...


Cross-Validation Progress:   0%|          | 0/10 [00:00<?, ?it/s]


--- Fold 1/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 1 - Test Loss: 1.6396, Test Gaze Acc: 0.6486, Test Laughter Acc: 0.5766

--- Fold 2/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 2 - Test Loss: 1.3142, Test Gaze Acc: 0.7598, Test Laughter Acc: 0.7477

--- Fold 3/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 3 - Test Loss: 1.5902, Test Gaze Acc: 0.6306, Test Laughter Acc: 0.5796

--- Fold 4/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 4 - Test Loss: 1.9413, Test Gaze Acc: 0.0300, Test Laughter Acc: 0.8679

--- Fold 5/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Trai

# Multi-LSTM

In [23]:
class EnfantReactionPredictorMultiLSTM(nn.Module):
    def __init__(self,
                 # Vocabularies for embedding lookups
                 vocabs,
                 # Whether to include gaze relation prediction
                 isGazeRelation,
                 # LSTM hyperparameters
                 hidden_dim = 192,
                 num_layers = 2,
                 dropout = 0.3,
                 input_size = 40,
                 bidirectional = False
                 ):
        super().__init__()

        self.isGazeRelation = isGazeRelation

        # Store modality vocabularies
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_gaze_chi = vocabs["gaze_chi"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # Embedding layers for categorical inputs
        self.embed_gaze = nn.Embedding(len(self.vocab_gaze), 8)
        self.embed_utterance = nn.Embedding(len(self.vocab_utterance), 8)
        self.embed_prosody = nn.Embedding(len(self.vocab_prosody), 4)
        self.embed_facial = nn.Embedding(len(self.vocab_facial), 8)
        self.embed_laughter = nn.Embedding(len(self.vocab_laughter), 4)

        # Sentence transformer for action text encoding
        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        # Embeddings for child laughter and gaze
        self.embed_laughter_chi = nn.Embedding(len(self.vocab_laughter_chi), 4)
        self.embed_gaze_chi = nn.Embedding(len(self.vocab_gaze_chi), 8)
        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # LSTM configuration for each modality
        self.hidden_dim = hidden_dim

        gaze_lstm_input_size = self.embed_gaze.embedding_dim
        utt_lstm_input_size = self.embed_utterance.embedding_dim
        prosody_lstm_input_size = self.embed_prosody.embedding_dim
        facial_lstm_input_size = self.embed_facial.embedding_dim
        laughter_lstm_input_size = self.embed_laughter.embedding_dim
        action_lstm_input_size = self.sentence_encoder.get_sentence_embedding_dimension()

        # One LSTM per modality
        self.lstm_gaze = nn.LSTM(gaze_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_utt = nn.LSTM(utt_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_prosody = nn.LSTM(prosody_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_facial = nn.LSTM(facial_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_laughter = nn.LSTM(laughter_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_action = nn.LSTM(action_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

        # Fusion layer after concatenating all modality features
        fusion_input_size = hidden_dim * 6 * (2 if bidirectional else 1)
        self.fc_fusion = nn.Linear(fusion_input_size, hidden_dim)

        self.dropout = nn.Dropout(dropout)

        # Output layers for each prediction target
        self.decoder_laughter = nn.Linear(hidden_dim, len(self.vocab_laughter_chi))
        self.decoder_gaze = nn.Linear(hidden_dim, len(self.vocab_gaze_chi))
        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(hidden_dim, len(self.vocab_gazerelation))

    # Finds the index of a value in the vocabulary (or <UNK> if missing)
    def lookup(self, vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    # Converts a batch of sequences of modality dictionaries into embeddings
    def encode_input_batch(self, batch_sequence_of_dicts, device):
        """
        Args:
            batch_sequence_of_dicts: List[List[Dict]]  # [batch_size, seq_len]
        Returns:
            tuple of modality tensors with shape [batch_size, seq_len, embedding_dim]
        """
        batch_size = len(batch_sequence_of_dicts)
        seq_len = len(batch_sequence_of_dicts[0])

        # Prepare lists to hold token indices/embeddings
        gaze_indices, utterance_indices, prosody_indices, facial_indices, laughter_indices, action_embeddings = [], [], [], [], [], []

        for sequence in batch_sequence_of_dicts:
            gaze_seq, utt_seq, prosody_seq, facial_seq, laughter_seq, action_seq = [], [], [], [], [], []

            for step_dict in sequence:
                gaze_seq.append(self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT")))
                utt_seq.append(self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT")))
                prosody_seq.append(self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT")))
                facial_seq.append(self.lookup(self.vocab_facial, step_dict.get("Facial@MOT")))
                laughter_seq.append(self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT")))

                action_text = step_dict.get("Action@MOT")
                if action_text is None or not isinstance(action_text, str):
                    action_text = "<UNK>"
                encoded = self.sentence_encoder.encode(action_text)  # Vector representation
                action_seq.append(torch.tensor(encoded))

            # Convert sequences to tensors
            gaze_indices.append(torch.tensor(gaze_seq))
            utterance_indices.append(torch.tensor(utt_seq))
            prosody_indices.append(torch.tensor(prosody_seq))
            facial_indices.append(torch.tensor(facial_seq))
            laughter_indices.append(torch.tensor(laughter_seq))
            action_embeddings.append(torch.stack(action_seq))

        # Embed categorical inputs and move to device
        gaze = self.embed_gaze(torch.stack(gaze_indices).to(device))
        utt = self.embed_utterance(torch.stack(utterance_indices).to(device))
        prosody = self.embed_prosody(torch.stack(prosody_indices).to(device))
        facial = self.embed_facial(torch.stack(facial_indices).to(device))
        laughter = self.embed_laughter(torch.stack(laughter_indices).to(device))
        action = torch.stack(action_embeddings).to(device)

        return gaze, utt, prosody, facial, laughter, action

    def forward(self, batch_sequence_of_dicts, device):
        """
        Forward pass:
        - Encodes each modality
        - Passes them through modality-specific LSTMs
        - Concatenates final hidden states
        - Passes through fusion + output layers
        """
        # Encode inputs
        gaze_emb, utt_emb, prosody_emb, facial_emb, laughter_emb, action_emb = self.encode_input_batch(batch_sequence_of_dicts, device)

        # Helper to get last hidden state from LSTM output
        def get_last_hidden(h_n, num_layers, bidirectional):
            if bidirectional:
                return torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
            else:
                return h_n[-1, :, :]

        # Process each modality with its LSTM and extract last hidden state
        _, (h_gaze, _) = self.lstm_gaze(gaze_emb)
        h_gaze_last = get_last_hidden(h_gaze, self.lstm_gaze.num_layers, self.lstm_gaze.bidirectional)

        _, (h_utt, _) = self.lstm_utt(utt_emb)
        h_utt_last = get_last_hidden(h_utt, self.lstm_utt.num_layers, self.lstm_utt.bidirectional)

        _, (h_prosody, _) = self.lstm_prosody(prosody_emb)
        h_prosody_last = get_last_hidden(h_prosody, self.lstm_prosody.num_layers, self.lstm_prosody.bidirectional)

        _, (h_facial, _) = self.lstm_facial(facial_emb)
        h_facial_last = get_last_hidden(h_facial, self.lstm_facial.num_layers, self.lstm_facial.bidirectional)

        _, (h_laughter, _) = self.lstm_laughter(laughter_emb)
        h_laughter_last = get_last_hidden(h_laughter, self.lstm_laughter.num_layers, self.lstm_laughter.bidirectional)

        _, (h_action, _) = self.lstm_action(action_emb)
        h_action_last = get_last_hidden(h_action, self.lstm_action.num_layers, self.lstm_action.bidirectional)

        # Concatenate final hidden states from all modalities
        h_final = torch.cat([h_gaze_last, h_utt_last, h_prosody_last, h_facial_last, h_laughter_last, h_action_last], dim=1)

        # Fusion layer + activation + dropout
        fused = self.dropout(torch.relu(self.fc_fusion(h_final)))

        # Prediction heads
        pred_laughter = self.decoder_laughter(fused)
        pred_gaze = self.decoder_gaze(fused)

        if self.isGazeRelation:
            pred_gazerelation = self.decoder_gazerelation(fused)
            return (pred_gaze, pred_laughter, pred_gazerelation)

        return (pred_gaze, pred_laughter)


In [31]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=False,
    num_layers=1,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 1
)

model = training_model(EnfantReactionPredictorMultiLSTM, dataset_params, model_params, train_params)

Starting training...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch [1/1], 	Train Loss: 1.9636, Train Gaze Acc: 0.5142, Train Gaze Prec: 0.2644, Train Gaze F1: 0.3492, Train Laughter Acc: 0.7399, Train Laughter Prec: 0.5474, Train Laughter F1: 0.6293, 
	Val Loss: 2.1311, Val Gaze Acc: 0.5363, Val Gaze Prec: 0.2876, Val Gaze F1: 0.3744, Val Laughter Acc: 0.5323, Val Laughter Prec: 0.2833, Val Laughter F1: 0.3698
Training finished.

Test Set Evaluation:
Test Loss: 2.3203, Test Gaze Acc: 0.4960, Test Gaze Prec: 0.2460, Test Gaze F1: 0.3289, Test Laughter Acc: 0.3690, Test Laughter Prec: 0.1362, Test Laughter F1: 0.1990
